In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
# reading all the words
words = open('cleaned_names.txt', 'r').read().splitlines()
words[:8]

['aaban', 'aabid', 'aabidah', 'aabir', 'aabriella', 'aada', 'aadam', 'aadarsh']

In [3]:
len(words)

29681

In [4]:
# building the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [5]:
# building the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1]) # 80% of the data
Xdev, Ydev = build_dataset(words[n1:n2]) # 10% of the data
Xte, Yte = build_dataset(words[n2:]) # 10% of the data

torch.Size([170379, 3]) torch.Size([170379])
torch.Size([21222, 3]) torch.Size([21222])
torch.Size([21124, 3]) torch.Size([21124])


In [47]:
# MLP revisited
n_embd = 10 # the dimensionality of the embeddings
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((vocab_size, n_embd),          generator=g) # the embedding matrix
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) # the first layer of the MLP
b1 = torch.randn((n_hidden),                    generator=g) # the bias of the first layer
W2 = torch.randn((n_hidden, vocab_size),        generator=g) * 0.01 # the second layer of the MLP
b2 = torch.randn((vocab_size),                  generator=g) * 0 # the bias of the second layer

parameters = [C, W1, b1, W2, b2]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True


11897


In [48]:
# same optimization as in part 2
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):

  # minibatch construction
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

  # forward pass
  emb = C[Xb] # embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
  hpreact = embcat @ W1 + b1 # hidden layer pre-activation
  h = torch.tanh(hpreact) # hidden layer
  logits = h @ W2 + b2 # output layer
  loss = F.cross_entropy(logits, Yb) # loss function

  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
  for p in parameters:
    p.data += -lr * p.grad

  # tracking the stats
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: loss = {loss.item():.4f}')
  # lossi.append(loss.log10.item())

      0/ 200000: loss = 3.3210
  10000/ 200000: loss = 2.0648
  20000/ 200000: loss = 2.4995
  30000/ 200000: loss = 2.2527
  40000/ 200000: loss = 1.9227
  50000/ 200000: loss = 2.4856
  60000/ 200000: loss = 2.7166
  70000/ 200000: loss = 2.2976
  80000/ 200000: loss = 2.3129
  90000/ 200000: loss = 2.6098
 100000/ 200000: loss = 2.1044
 110000/ 200000: loss = 2.1216
 120000/ 200000: loss = 2.0399
 130000/ 200000: loss = 2.4138
 140000/ 200000: loss = 2.0592
 150000/ 200000: loss = 2.2273
 160000/ 200000: loss = 2.1260
 170000/ 200000: loss = 2.1158
 180000/ 200000: loss = 2.1369
 190000/ 200000: loss = 2.2586


In [50]:
logits[0] # logits should be much much closer to zero

tensor([ 4.5756,  2.1158, -1.3267,  0.6512, -0.8854,  2.8523, -0.2493, -2.2145,
        -1.5781,  1.1665, -0.7887,  0.8991,  0.2655, -0.7051,  0.5133, -0.2217,
        -1.5224, -1.2577,  0.3792,  1.9003,  0.5740, -0.4448,  0.0906, -3.5888,
        -1.8558,  2.8218, -0.5220], grad_fn=<SelectBackward0>)

In [51]:
# probability for any character
-torch.tensor(1/27.0).log()

tensor(3.2958)

In [52]:
# 4 dimensional example of the issue
logits = torch.tensor([1.0, 1.0, 1.0, 1.0])
# logits = torch.randn(4) * 10
probs = torch.softmax(logits, dim=0)
loss = -probs[2].log()
logits, probs, loss

(tensor([1., 1., 1., 1.]),
 tensor([0.2500, 0.2500, 0.2500, 0.2500]),
 tensor(1.3863))

In [24]:
# plt.plot(lossi)

In [53]:
@torch.no_grad() # this ensures that we don't save gradients
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte)
  }[split]
  emb = C[x] # (N, block_size, n_embd) embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors (N, block_size * n_embd)
  h = torch.tanh(embcat @ W1 + b1) # hidden layer (N, n_hidden)
  logits = h @ W2 + b2 # output layer (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())
  
split_loss('train')
split_loss('val')
split_loss('test')

train 2.0761170387268066
val 2.1445398330688477
test 2.158021926879883


In [54]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10) # for reproducibility

for _ in range(20):

  out = []
  context = [0] * block_size # initialize with all dots
  while True:
    # forward pass
    emb = C[torch.tensor([context])] # embedding
    h = torch.tanh(emb.view(1, -1) @ W1 + b1) # hidden layer
    logits = h @ W2 + b2 # output layer
    probs = F.softmax(logits, dim=1) # probabilities
    
    # sample from the distribution
    ix = torch.multinomial(probs, num_samples=1, generator=g).item()
    
    # shift the context window and track the sample
    context = context[1:] + [ix]
    out.append(ix)
    
    # stop if we reach the end
    if ix == 0:
      break

  print(''.join(itos[i] for i in out)) # decode and print the sample

mora.
kayanniee.
med.
rylee.
emmadiendra.
grace.
korelin.
shy.
jen.
eden.
van.
aarvelyzion.
kalin.
shubvigahimies.
kin.
ron.
jerenterianuel.
gedt.
kylene.
ehs.
